In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY_4')
azure_openai_endpoint = os.getenv('AZURE_OPENAI_API_ENDPOINT_4')
deployment_name = os.getenv('AZURE_DEPLOYMENT_NAME_4')


# Set up Azure OpenAI API (for Azure deployment)
openai.api_type = "azure"
openai.api_key = azure_openai_api_key
openai.api_base = azure_openai_endpoint
openai.api_version = "2023-05-15"  # Ensure this is up to date

import warnings
warnings.filterwarnings('ignore')

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
import pandas as pd

In [3]:
llm = AzureChatOpenAI(
    api_key=azure_openai_api_key,  # Azure OpenAI API Key
    api_version="2023-12-01-preview",  # API version (you can adjust it as needed)
    azure_endpoint=azure_openai_endpoint,  # Azure OpenAI endpoint
    deployment_name=deployment_name,  # Ensure deployment_name is the correct one you configured in Azure
    temperature=0  # Optional: Adjust the temperature for randomness in responses
)

C:\Users\antoa\AppData\Local\Temp\ipykernel_50208\1637258887.py:1: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(


Ce processus est utile pour préparer des données textuelles structurées à partir d'un CSV en tant qu'objets structurés avec des métadonnées, ce qui est souvent nécessaire pour le traitement ultérieur ou l'analyse de texte.

In [4]:

# Charger le CSV
csv_path = "../../RGBD/table_produits/produits.csv"   # Remplace avec le chemin réel si besoin
df = pd.read_csv(csv_path, encoding='utf-8')

# Vérifier que la colonne "description" existe bien
if "description" not in df.columns:
    raise ValueError("La colonne 'description' n'existe pas dans le CSV ! Vérifie le nom des colonnes.")
# Vérification de la colonne "id_produit"
if "id_produit" not in df.columns:
    raise ValueError("La colonne 'id_produit' n'existe pas dans le CSV ! Vérifiez le nom des colonnes.")

# Transformer chaque ligne en objet Document avec métadonnées
documents = []
for index, row in df.iterrows():
    metadata = {"source": csv_path, "row": index,"id_produit": row["id_produit"]}

    if "saveur" in row:
        metadata["saveur"] = str(row["saveur"])  # Ajouter la saveur dans les métadonnées

    documents.append(Document(page_content=str(row["description"]), metadata=metadata))

print(f"Nombre de documents préparés : {len(documents)}")

Nombre de documents préparés : 1531


ce code te permet de transformer tes documents en embeddings vectoriels et de les sauvegarder dans un format FAISS, prêt à être utilisé pour des recherches ou des analyses futures sur tes données textuelles.

In [59]:
# Étape 3: Création et sauvegarde du vecteur FAISS avec métadonnées
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
try:
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local('faiss_vector_store')
    print("Vector store enregistré avec succès !")
except Exception as e:
    print(f"Erreur lors de la création du vectorstore : {e}")

Vector store enregistré avec succès !


In [82]:
query = "Quel produit contient des saveurs de fraise et bonbon ?"

In [88]:
import re

# Fonction pour extraire les saveurs de la requête et les rechercher dans les résultats
def extract_flavors_from_query(query):
    """
    Cette fonction extrait les saveurs présentes dans la requête `query`
    et retourne une liste des saveurs trouvées parmi celles définies dans `flavor_list`.
    """
    # Liste des saveurs disponibles
    flavor_list = ["cassis", "lime", "fraise", "menthe", "mangue", "pêche", "bonbon"]

    found_flavors = []
    
    # Pour chaque saveur de la liste, vérifier si elle est mentionnée dans la requête
    for flavor in flavor_list:
        if re.search(rf'\b{flavor}\b', query, re.IGNORECASE):
            found_flavors.append(flavor)

    return found_flavors


# Fonction pour interroger le vecteur et récupérer les saveurs associées à la requête
def search_for_flavor(query):
    # Extraire les saveurs mentionnées dans la requête
    flavors_in_query = extract_flavors_from_query(query)

    # Afficher uniquement les saveurs extraites
    print(f"Saveurs extraites de la requête : {flavors_in_query}")

    # Retourner les saveurs extraites sous forme de chaîne, séparées par des virgules
    return ", ".join(flavors_in_query)

# Exemple d'appel à la fonction avec une requête utilisateur





ce code permet de charger un vector store FAISS sauvegardé et de configurer un retriever pour effectuer des recherches dans ce store en utilisant un seuil de similarité pour déterminer la pertinence des résultats. Cela est utile lorsque tu veux interroger tes documents en fonction de leur similarité par rapport à une requête donnée.

In [97]:
# Chargement du vecteur
vectorstore = FAISS.load_local('faiss_vector_store', embeddings=embeddings, allow_dangerous_deserialization=True)

retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.3}
)

In [92]:
query = "Quel produit contient des saveurs de fraise et bonbon ?"

In [93]:

# Utilisation de la fonction search_for_flavor dans retriever.invoke
flavors_in_query = search_for_flavor(query)

Saveurs extraites de la requête : ['fraise', 'bonbon']


In [94]:
flavors_in_query

'fraise, bonbon'

In [98]:
retriever.invoke(query)

[Document(metadata={'source': '../../RGBD/table_produits/produits.csv', 'row': 853, 'id_produit': 855, 'saveur': 'parfait, pomme, bonbon, bonbons'}, page_content="Le e-liquide Pomme Candy reprend tout ce qu'on attend d'ungoût bonbon: un fruit particulièrement alléchant, mais également juteux, frais et un peu acidulé, et une base de friandise ultra gourmande et sucrée ! Minutieusement préparé par Prestige Fruits, en France, ce e liquide est disponible en versionnon nicotinée et surboostée de 50ml. Sa fiole 60ml pourra accueillir un booster de nicotine (non inclus). Un bonbon à la pomme, juste comme on l'aime Vous aimez quand la vape est puissante et sucrée ? Il vous faut donc vous tourner en priorité vers les saveurs bonbons frais et fruités ! Ici, le fabricant français Prestige Fruits vous propose un mix depommestrès intéressant. En plus des notes typiques de ces fruits du verger, Prestige Fruits ajoute une bonne dose d'acidité et de sucre, pour une tendancefriandisebien assumée... Un 

In [99]:
retriever.invoke(query, filter={"saveur": flavors_in_query})

[Document(metadata={'source': '../../RGBD/table_produits/produits.csv', 'row': 842, 'id_produit': 844, 'saveur': 'fraise, bonbon'}, page_content="Le Fraise Candy est un e liquide 50ml d'origine française, fabriqué par Prestige Fruits (Vape Connection) pour la gamme Sweety Fruits. Il vous propose une saveur fraîche de fraise façon bonbon. Créé à partir d'arômes, de propylène glycol (PG) et de glycérine végétale (VG), il est proposé en50PG/50VGafin d'offrir un excellent compromis entre saveurs, hit en gorge et vapeur. Il est compatible tout type de matériel e-cig. Le Fraise Candy est un e liquide surboosté et non nicotiné, conditionné en fiole de type PET 60ml. Dessus, un bouchon avec sécurité enfant et un embout fin très pratique. Rempli à hauteur de 50ml, il pourra être accompagné d'un booster de nicotine au besoin. Qui est Prestige Fruits ? Comme son nom l'indique, le fabriquant française s'est spécialisé dans la reproduction des saveurs de fruits à travers sa large gamme d'e-liquides

In [69]:


# Paramètres de la recherche sans 'score_threshold' et changement du search_type
search_kwargs = {
    "k": 5,                  # Retourne les 5 meilleurs résultats
    "nprobe": 2              # Recherche sur un nombre de clusters plus grand pour améliorer les résultats
}

# Initialisation du retriever sans 'score_threshold' et en utilisant un search_type adapté
retriever = vectorstore.as_retriever(
    search_type="similarity",  # Utiliser un type de recherche basé sur la similarité, sans score_threshold
    search_kwargs=search_kwargs
)

In [ ]:
flavors_in_query

'fraise'

In [71]:
retriever.invoke(query)

[Document(metadata={'source': '../../RGBD/table_produits/produits.csv', 'row': 795, 'id_produit': 797, 'saveur': 'fraise, lime, bonbon, fruit du dragon'}, page_content='Le Freho Extra Frais 50ml est un e-liquide à booster fabriqué en France par la marque e.Tasty. Il vous propose des saveurs fraîches façon bonbon à la fraise et fruit du dragon. Sa recette est composée d\'arômes alimentaires, de propylène glycol (PG) et de glycérine végétale (VG), pour un ratio50PG/50VG. Sa recette pourra au besoin êtrecomplétée avec des boosters10ml de nicotine (non inclus). Il vous est livré dans un flacon de 75ml doté d\'un bouchon sécurisé et d\'un embout fin compte-gouttes. Qui est e.Tasty ? e.Tasty est un fabricant français reconnu dans le monde de la vape pour sa maîtrise des saveurs, mais également pour sa recherche constante de qualité supérieure. Ainsi, qu\'elles soient fruitées, fraîches ou encore classics, soyez toujours certain avec e.Tasty de vapoter des recettes de qualité premium ! Quelqu

In [73]:
retriever.invoke(query, filter={"saveur": flavors_in_query})

[]

In [72]:
retriever.invoke(query, filter={"saveur": "fraise, bonbon"})

[Document(metadata={'source': '../../RGBD/table_produits/produits.csv', 'row': 842, 'id_produit': 844, 'saveur': 'fraise, bonbon'}, page_content="Le Fraise Candy est un e liquide 50ml d'origine française, fabriqué par Prestige Fruits (Vape Connection) pour la gamme Sweety Fruits. Il vous propose une saveur fraîche de fraise façon bonbon. Créé à partir d'arômes, de propylène glycol (PG) et de glycérine végétale (VG), il est proposé en50PG/50VGafin d'offrir un excellent compromis entre saveurs, hit en gorge et vapeur. Il est compatible tout type de matériel e-cig. Le Fraise Candy est un e liquide surboosté et non nicotiné, conditionné en fiole de type PET 60ml. Dessus, un bouchon avec sécurité enfant et un embout fin très pratique. Rempli à hauteur de 50ml, il pourra être accompagné d'un booster de nicotine au besoin. Qui est Prestige Fruits ? Comme son nom l'indique, le fabriquant française s'est spécialisé dans la reproduction des saveurs de fruits à travers sa large gamme d'e-liquides

In [159]:
vectorstore = FAISS.load_local('faiss_vector_store', embeddings=embeddings, allow_dangerous_deserialization=True)

retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
     search_kwargs={
        "score_threshold": 0.3,  # Seuillage du score de similarité
        "k": 5,                  # Retourne les 5 meilleurs résultats
        "nprobe": 2,            # Augmente la recherche pour trouver plus de voisins
    },
    search_params={
        "metric": "cosine",        # Utilise la distance cosine plutôt que euclidienne
        "efSearch": 128            # Ajuste l'efficacité de la recherche FAISS
    }
)

In [160]:
retriever.invoke(query)

[Document(metadata={'source': '../../RGBD/table_produits/produits.csv', 'row': 795, 'id_produit': 797, 'saveur': 'fraise, lime, bonbon, fruit du dragon'}, page_content='Le Freho Extra Frais 50ml est un e-liquide à booster fabriqué en France par la marque e.Tasty. Il vous propose des saveurs fraîches façon bonbon à la fraise et fruit du dragon. Sa recette est composée d\'arômes alimentaires, de propylène glycol (PG) et de glycérine végétale (VG), pour un ratio50PG/50VG. Sa recette pourra au besoin êtrecomplétée avec des boosters10ml de nicotine (non inclus). Il vous est livré dans un flacon de 75ml doté d\'un bouchon sécurisé et d\'un embout fin compte-gouttes. Qui est e.Tasty ? e.Tasty est un fabricant français reconnu dans le monde de la vape pour sa maîtrise des saveurs, mais également pour sa recherche constante de qualité supérieure. Ainsi, qu\'elles soient fruitées, fraîches ou encore classics, soyez toujours certain avec e.Tasty de vapoter des recettes de qualité premium ! Quelqu

In [161]:
retriever.invoke(query, filter={"saveur": flavors_in_query})

No relevant docs were retrieved using the relevance score threshold 0.3


[]

In [105]:
retriever = vectorstore.as_retriever(
    search_type="similarity",  # Recherche des voisins les plus proches
    search_kwargs={
        "k": 3,  # Retourne les 3 voisins les plus proches
    }
)

In [107]:
retriever.invoke(query)

[Document(metadata={'source': '../../RGBD/table_produits/produits.csv', 'row': 723, 'saveur': 'cassis, lime'}, page_content="Le Cassis est un e-liquide fabriqué en France par Curieux pour sa collection Natural. Il vous propose unesaveur fraîche de cassis. Sa recette à booster et sans nicotine se compose d'arômes alimentaires, de végétol (50%) et de glycérine végétale (50%). Elle pourra au besoin êtrepersonnaliséeavec de la base neutre ou des boosters 10ml de nicotine (non inclus). Le flacon de 70ml du e-liquide Cassis 50ml est doté d'un bouchon sécurisé et d'un embout fin facilitant le remplissage de votre réservoir. Qui est Curieux ? Curieux est un fabricant français de liquides pour cigarette électronique, réputé notamment pour sa célèbre recette fruitée du Licorne. La marque décline ses recettes en différentes gammes, permettant à tout vapoteur de trouver le mix qui lui convient : Editions Astrale, Natural ou encore Oldies. Retrouvez les créations de chez Curieux en format e-liquide

In [106]:
retriever.invoke(query, filter={"saveur": flavors_in_query})

[Document(metadata={'source': '../../RGBD/table_produits/produits.csv', 'row': 723, 'saveur': 'cassis, lime'}, page_content="Le Cassis est un e-liquide fabriqué en France par Curieux pour sa collection Natural. Il vous propose unesaveur fraîche de cassis. Sa recette à booster et sans nicotine se compose d'arômes alimentaires, de végétol (50%) et de glycérine végétale (50%). Elle pourra au besoin êtrepersonnaliséeavec de la base neutre ou des boosters 10ml de nicotine (non inclus). Le flacon de 70ml du e-liquide Cassis 50ml est doté d'un bouchon sécurisé et d'un embout fin facilitant le remplissage de votre réservoir. Qui est Curieux ? Curieux est un fabricant français de liquides pour cigarette électronique, réputé notamment pour sa célèbre recette fruitée du Licorne. La marque décline ses recettes en différentes gammes, permettant à tout vapoteur de trouver le mix qui lui convient : Editions Astrale, Natural ou encore Oldies. Retrouvez les créations de chez Curieux en format e-liquide

En résumé, ce code permet de rechercher des saveurs spécifiées dans une requête utilisateur et de retourner une liste de ces saveurs pour les utiliser dans une recherche plus large, par exemple dans un système de récupération d'informations basé sur des embeddings ou des vecteurs.

In [111]:
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
     search_kwargs={
        "score_threshold": 0.3,  # Seuillage du score de similarité
        "k": 5,                  # Retourne les 5 meilleurs résultats
        "nprobe": 10,            # Augmente la recherche pour trouver plus de voisins
    },
    search_params={
        "metric": "cosine",        # Utilise la distance cosine plutôt que euclidienne
        "efSearch": 128            # Ajuste l'efficacité de la recherche FAISS
    }
)

In [112]:
retriever.invoke(query)

[Document(metadata={'source': '../../RGBD/table_produits/produits.csv', 'row': 723, 'saveur': 'cassis, lime'}, page_content="Le Cassis est un e-liquide fabriqué en France par Curieux pour sa collection Natural. Il vous propose unesaveur fraîche de cassis. Sa recette à booster et sans nicotine se compose d'arômes alimentaires, de végétol (50%) et de glycérine végétale (50%). Elle pourra au besoin êtrepersonnaliséeavec de la base neutre ou des boosters 10ml de nicotine (non inclus). Le flacon de 70ml du e-liquide Cassis 50ml est doté d'un bouchon sécurisé et d'un embout fin facilitant le remplissage de votre réservoir. Qui est Curieux ? Curieux est un fabricant français de liquides pour cigarette électronique, réputé notamment pour sa célèbre recette fruitée du Licorne. La marque décline ses recettes en différentes gammes, permettant à tout vapoteur de trouver le mix qui lui convient : Editions Astrale, Natural ou encore Oldies. Retrouvez les créations de chez Curieux en format e-liquide

In [113]:
retriever.invoke(query, filter={"saveur": flavors_in_query})

[Document(metadata={'source': '../../RGBD/table_produits/produits.csv', 'row': 723, 'saveur': 'cassis, lime'}, page_content="Le Cassis est un e-liquide fabriqué en France par Curieux pour sa collection Natural. Il vous propose unesaveur fraîche de cassis. Sa recette à booster et sans nicotine se compose d'arômes alimentaires, de végétol (50%) et de glycérine végétale (50%). Elle pourra au besoin êtrepersonnaliséeavec de la base neutre ou des boosters 10ml de nicotine (non inclus). Le flacon de 70ml du e-liquide Cassis 50ml est doté d'un bouchon sécurisé et d'un embout fin facilitant le remplissage de votre réservoir. Qui est Curieux ? Curieux est un fabricant français de liquides pour cigarette électronique, réputé notamment pour sa célèbre recette fruitée du Licorne. La marque décline ses recettes en différentes gammes, permettant à tout vapoteur de trouver le mix qui lui convient : Editions Astrale, Natural ou encore Oldies. Retrouvez les créations de chez Curieux en format e-liquide

In [115]:
retriever = vectorstore.as_retriever(
    search_type="similarity",  # Recherche des voisins les plus proches
    search_kwargs={
        "k": 3,  # Retourne les 3 voisins les plus proches
    }
)

In [116]:
# Requête de recherche
query = "Quel produit contient des saveurs de cassis ?"
results = retriever.invoke(query)

# Extraction des métadonnées des résultats
for result in results:
    print(f"Contenu : {result.page_content}")
    print(f"Saveur : {result.metadata.get('saveur', 'Aucune saveur spécifiée')}") 

Contenu : Le Cassis Mangue est un e-liquideà boosterde la collection Fruizee par Eliquid France, auxsaveurs fraîches de mangue et de cassis. Sa recette fabriquée en France se compose d'arômes alimentaires, de propylène glycol (PG) et de glycérine végétale (VG), pour un ratio 30PG/70VG. Vous pouvez au besoin ajouter un booster 10ml de nicotine (non inclus). Il vous est livré dans unflacon de 60mldoté d'un embout fin et d'un bouchon avec sécurité enfant. Qui est Eliquid France ? Créé en 2013 à Nieul sur Mer en Charente-Maritime, le laboratoire Eliquid France s'est fait une place de choix dans l'univers de la vape grâce à des liquides de qualité, exportant ses créations dans le monde entier. Toujours en recherche d'innovation, la marque française propose des recettes variées aux saveurs allant de simples à complexes, et se déclinant autant en arôme DIY qu'en e-liquide prêt à vaper. La gamme Fruizee Collection ayant fait la réputation de la marque Eliquid France, elle mélange des saveurs f

In [117]:
# Requête de recherche
query = "Quel produit contient des saveurs de cassis ?"
results = retriever.invoke(query, filter={"saveur": flavors_in_query})

# Extraction des métadonnées des résultats
for result in results:
    print(f"Contenu : {result.page_content}")
    print(f"Saveur : {result.metadata.get('saveur', 'Aucune saveur spécifiée')}") 

Contenu : Le Cassis est un e-liquide fabriqué en France par Curieux pour sa collection Natural. Il vous propose unesaveur fraîche de cassis. Sa recette à booster et sans nicotine se compose d'arômes alimentaires, de végétol (50%) et de glycérine végétale (50%). Elle pourra au besoin êtrepersonnaliséeavec de la base neutre ou des boosters 10ml de nicotine (non inclus). Le flacon de 70ml du e-liquide Cassis 50ml est doté d'un bouchon sécurisé et d'un embout fin facilitant le remplissage de votre réservoir. Qui est Curieux ? Curieux est un fabricant français de liquides pour cigarette électronique, réputé notamment pour sa célèbre recette fruitée du Licorne. La marque décline ses recettes en différentes gammes, permettant à tout vapoteur de trouver le mix qui lui convient : Editions Astrale, Natural ou encore Oldies. Retrouvez les créations de chez Curieux en format e-liquide prêt-à-vaper, à booster, ou en concentré DIY ! La gamme Natural Le fabricant français vous propose une collection 

In [118]:
# Requête de recherche
query = "Quel produit contient des saveurs de cassis ?"
results = retriever.get_relevant_documents(query, filter={"saveur": flavors_in_query})

# Extraction des métadonnées des résultats
for result in results:
    print(f"Contenu : {result.page_content}")
    print(f"Saveur : {result.metadata.get('saveur', 'Aucune saveur spécifiée')}") 

Contenu : Le Cassis est un e-liquide fabriqué en France par Curieux pour sa collection Natural. Il vous propose unesaveur fraîche de cassis. Sa recette à booster et sans nicotine se compose d'arômes alimentaires, de végétol (50%) et de glycérine végétale (50%). Elle pourra au besoin êtrepersonnaliséeavec de la base neutre ou des boosters 10ml de nicotine (non inclus). Le flacon de 70ml du e-liquide Cassis 50ml est doté d'un bouchon sécurisé et d'un embout fin facilitant le remplissage de votre réservoir. Qui est Curieux ? Curieux est un fabricant français de liquides pour cigarette électronique, réputé notamment pour sa célèbre recette fruitée du Licorne. La marque décline ses recettes en différentes gammes, permettant à tout vapoteur de trouver le mix qui lui convient : Editions Astrale, Natural ou encore Oldies. Retrouvez les créations de chez Curieux en format e-liquide prêt-à-vaper, à booster, ou en concentré DIY ! La gamme Natural Le fabricant français vous propose une collection 

rag

In [119]:

# Définition du prompt pour la génération de réponse
prompt_template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.

Question: {question}

Context: {context}

Answer:
"""

# Initialisation du prompt
prompt = ChatPromptTemplate.from_template(prompt_template)

# Formatage des documents de contexte
def format_docs(flavors):
    # Création du contexte avec les saveurs extraites
    return f"Les saveurs extraites sont : {flavors}"

# Pipeline de question-réponse
qa_chain = (
    {
        "context": retriever | format_docs,  # Utilisation du retriever pour obtenir des contextes
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()  # Extraction de la réponse
)

# Exécution de la chaîne de question-réponse et affichage du résultat
# Ici, tu passes les saveurs extraites en tant que contexte dans le modèle
response = qa_chain.invoke(query, filter={"saveur": flavors_in_query})
# Affichage du résultat
print(f"Réponse générée par le modèle : {response}")

Réponse générée par le modèle : Les produits qui contiennent des saveurs de cassis sont :

1. Le Cassis Mangue, un e-liquide de la collection Fruizee par Eliquid France, qui combine les saveurs de mangue et de cassis.

2. Le Cassis, un e-liquide fabriqué par Curieux pour sa collection Natural, qui propose une saveur fraîche de cassis.

3. Le Cassis, un e-liquide de la collection Evolution par Liquideo, qui propose des saveurs de cassis.


https://www.aromes-et-liquides.fr/e-liquide-vampire-vape/17833-heisenberg-gum-ice.html,Heisenberg Gum Ice Vampire Vape,https://assets.aromes-et-liquides.fr/54364-thickbox_default/heisenberg-gum-ice.jpg,5.5,10ml,50/50,Royaume-Uni,Oui,Non,"baies, rose, baie, menthol, anis, lime","Le e-liquide Heisenberg Gum Ice est fabriqué au Royaume-Uni par Vampire Vape. Il reprend la recette originelle du Heisenberg en y additionnant un goût de chewing-gum rose. Il se compose d'arômes alimentaires,de propylène glycol (PG) et de glycérine végétale (VG), pour unratio PG/VG de 50/50. Ce e-liquide pour cigarette électronique est prêt à vaper (PAV), et élaboré dans une formule déjà nicotinée (taux au choix). Vous recevez le Heisenberg Gum Ice dans une fiole PET de 10ml, pourvue d'un bouchon avec sécurité enfant et d'un embout fin facilitant le remplissage de votre réservoir. Qui est Vampire Vape ? Le fabricant anglais s'est fait reconnaître dans le monde de la vape à travers ses nombreux succès misant principalement sur des saveurs fruitées, avec notamment la recette du Heisenberg. Dépassant à chaque fois leurs limites, les aromaticiens de la marque déclinent leur best-seller dans une version revisitée, le Heisenberg. Quelle est la recette originelle du Heisenberg ? Le Heisenberg est une création signée Vampire Vape. Sa recette mêle les saveurs fraîches et fruitées de baies des bois, d'anis et de menthol. Le fabricant anglais vous propose une gamme de e-liquides basée sur ce mix mentholé et anisé, en l'accompagnant de divers goûts fruités. Nos conseils pour utiliser votre e-liquide Heisenberg Gum L'utilisation d'un e-liquide est exclusivement réservée à la cigarette électronique. Nous vous recommandons de bien agiter votre flacon de e-liquide avant emploi. Pour limiter les risques de fuites et prolonger la durée de vie de vos résistances, nous vous conseillons d'utiliser un clearomiseur adapté au taux PG/VG de votre e-liquide. Quel taux de nicotine choisir pour votre e-liquide ? Le choix de votre taux de nicotine dépend de votre profil de fumeur ainsi que de la cigarette électronique utilisée. Si vous utilisez une cigarette électronique équipée d'une résistance égale ou supérieure à 1 ohm et à une puissance ne dépassant pas les 30 watts, voici nos recommandations pour choisir votre taux de nicotine : Si vous utilisez une cigarette électronique plus performante équipée de résistances dont la valeur est inférieure à 1 ohm et pour une puissance supérieure à 30 watts, nous vous conseillons d'utiliser un taux de nicotine plus faible : Conservation de votre e-liquide Pour une conservation optimale de votre e-liquide, nous vous conseillons de stocker votre flacon à l'abri de la lumière dans un droit sec et à température ambiante. Après contacts répétés à l'air et à la lumière, la couleur de votre e-liquide peut se foncer. Ce changement de couleur n'affecte ni la qualité ni le goût de votre e-liquide. La date indiquée sur l'étiquette est une DLUO (date limite d'utilisation optimale) ou DDM (date de durabilité minimale). Passé cette date, le e-liquide peut encore être consommé. Mises en garde et contre-indications : Précautions d'emploi :",Vampire Vape,fruit,"Le fabricant anglais s'est fait reconnaître dans le monde de la vape à travers ses nombreux succès misant principalement sur des saveurs fruitées, avec notamment la recette du Heisenberg ou encore du Pinkman.",723

In [12]:
from langchain.document_loaders import CSVLoader

file_path = "C:/Users/antoa/Desktop/projet certif/app/rag_csv.csv"
loader = CSVLoader(file_path=file_path, encoding="utf-8")
documents = loader.load()

# Vérifier un document pour voir ses métadonnées
print(documents[0])  # Affiche le premier document


page_content='index: 0
url: https://www.aromes-et-liquides.fr/e-liquide-ultimate/9197-e-liquide-ultimate-ragnarok-par-al.html
nom_produit: Ragnarok Ultimate A&L
img_produit: https://assets.aromes-et-liquides.fr/53645-thickbox_default/e-liquide-ultimate-ragnarok-par-al.jpg
prix_produit: 5.9
contenance: 10ml
pg_vg: 50/50
origine: France
frais: Oui
surbooste: Non
saveur: cassis, lime, fruits rouges, parfait
description: Le e-liquide Ragnarok par A&L Ultimate est un produitprêt à l'emploipour cigarette électronique. Dans sa fiole 10ml, vous retrouverez des saveurs de fruits rouges, accompagnées d'un goût frais de cassis. Idéal pour débuter, ce e-liquide vous est proposé en différents taux de nicotine au choix : 0, 3, 6, ou 12mg/ml. Qu'est-ce que le Ragnarok Ultimate ? Le Ragnarok, c'est unerecette fraiche aux goûts de fruitsdehaute qualité française, signée Arômes et Liquides (A&L). Ici, vous en profiterez dans sa version e-liquide prêt à vaper. Conforme à la loi TPD, le e liquide Ragnarok

In [13]:
import pandas as pd
from langchain.schema import Document

file_path = "C:/Users/antoa/Desktop/projet certif/app/rag_csv.csv"

# Charger le CSV
df = pd.read_csv(file_path, encoding="utf-8")

# Vérifier les colonnes disponibles
print(df.columns)

# Créer les Documents avec les métadonnées
documents = [
    Document(
        page_content=row["description"] if "description" in df.columns else "",
        metadata={"id_produit": row["id_produit"], "saveur": row["saveur"]}
    )
    for _, row in df.iterrows()
]

# Vérifier un document pour voir ses métadonnées
print(documents[0])


Index(['index', 'url', 'nom_produit', 'img_produit', 'prix_produit',
       'contenance', 'pg_vg', 'origine', 'frais', 'surbooste', 'saveur',
       'description', 'brand', 'gout', 'info_brand', 'id_produit'],
      dtype='object')
page_content='Le e-liquide Ragnarok par A&L Ultimate est un produitprêt à l'emploipour cigarette électronique. Dans sa fiole 10ml, vous retrouverez des saveurs de fruits rouges, accompagnées d'un goût frais de cassis. Idéal pour débuter, ce e-liquide vous est proposé en différents taux de nicotine au choix : 0, 3, 6, ou 12mg/ml. Qu'est-ce que le Ragnarok Ultimate ? Le Ragnarok, c'est unerecette fraiche aux goûts de fruitsdehaute qualité française, signée Arômes et Liquides (A&L). Ici, vous en profiterez dans sa version e-liquide prêt à vaper. Conforme à la loi TPD, le e liquide Ragnarok est présenté en fiole PET de 10ml. Pourquoi ? Afin de vous assurer unproduit déjà nicotiné à votre convenance! Pourquoi choisir le Ragnarok en e-liquide 10ml ? Simple, pratiq

In [19]:
import pandas as pd
from langchain.document_loaders import CSVLoader
# Charger les données avec pandas pour récupérer les colonnes nécessaires

 
# Charger les données avec pandas pour récupérer les colonnes
df = pd.read_csv('rag_csv.csv', encoding="utf-8")


# Charger avec CSVLoader
loader = CSVLoader(file_path='rag_csv.csv', encoding='utf-8')
documents = loader.load()

# Ajouter les métadonnées à chaque document
for doc, (_, row) in zip(documents, df.iterrows()):
    doc.metadata["id_produit"] = row["id_produit"]
    doc.metadata["saveur"] = row["saveur"]

# Vérifier les métadonnées du premier document
print(documents[0])


page_content='index: 0
url: https://www.aromes-et-liquides.fr/e-liquide-ultimate/9197-e-liquide-ultimate-ragnarok-par-al.html
nom_produit: Ragnarok Ultimate A&L
img_produit: https://assets.aromes-et-liquides.fr/53645-thickbox_default/e-liquide-ultimate-ragnarok-par-al.jpg
prix_produit: 5.9
contenance: 10ml
pg_vg: 50/50
origine: France
frais: Oui
surbooste: Non
saveur: cassis, lime, fruits rouges, parfait
description: Le e-liquide Ragnarok par A&L Ultimate est un produitprêt à l'emploipour cigarette électronique. Dans sa fiole 10ml, vous retrouverez des saveurs de fruits rouges, accompagnées d'un goût frais de cassis. Idéal pour débuter, ce e-liquide vous est proposé en différents taux de nicotine au choix : 0, 3, 6, ou 12mg/ml. Qu'est-ce que le Ragnarok Ultimate ? Le Ragnarok, c'est unerecette fraiche aux goûts de fruitsdehaute qualité française, signée Arômes et Liquides (A&L). Ici, vous en profiterez dans sa version e-liquide prêt à vaper. Conforme à la loi TPD, le e liquide Ragnarok

In [20]:
retriever.invoke(query)

[Document(metadata={'source': '../../RGBD/table_produits/produits.csv', 'row': 795, 'id_produit': 797, 'saveur': 'fraise, lime, bonbon, fruit du dragon'}, page_content='Le Freho Extra Frais 50ml est un e-liquide à booster fabriqué en France par la marque e.Tasty. Il vous propose des saveurs fraîches façon bonbon à la fraise et fruit du dragon. Sa recette est composée d\'arômes alimentaires, de propylène glycol (PG) et de glycérine végétale (VG), pour un ratio50PG/50VG. Sa recette pourra au besoin êtrecomplétée avec des boosters10ml de nicotine (non inclus). Il vous est livré dans un flacon de 75ml doté d\'un bouchon sécurisé et d\'un embout fin compte-gouttes. Qui est e.Tasty ? e.Tasty est un fabricant français reconnu dans le monde de la vape pour sa maîtrise des saveurs, mais également pour sa recherche constante de qualité supérieure. Ainsi, qu\'elles soient fruitées, fraîches ou encore classics, soyez toujours certain avec e.Tasty de vapoter des recettes de qualité premium ! Quelqu

précision 
La précision mesure la proportion des éléments recommandés qui sont effectivement pertinents pour l'utilisateur. C'est-à-dire, parmi toutes les recommandations faites par le modèle, quelle part est réellement appréciée par l'utilisateur ou est pertinente par rapport à ses attentes.

In [38]:
csv_path = "../../RGBD/table_produits/produits.csv"   # Remplace avec le chemin réel si besoin
df = pd.read_csv(csv_path, encoding='utf-8')

In [39]:
df.shape

(1531, 15)

In [123]:
retriever = vectorstore.as_retriever(
        search_type="similarity",  # Recherche des voisins les plus proches
        search_kwargs={
            "k": 10,  # Retourner les 10 voisins les plus proches
        }
    )

In [124]:
# Fonction pour récupérer les recommandations pertinentes (par exemple, par ID ou feedback utilisateur)
def get_relevant_products(user_preferences):
    # Cette fonction retourne une liste d'IDs de produits pertinents (simulation ici)
    return user_preferences  # Liste des produits pertinents pour l'utilisateur

In [125]:
# Utilisez str.contains pour filtrer les lignes
# Filtrer les lignes où la colonne 'saveur' contient 'lime'
filtered_produit_id = df[df['saveur'].str.contains('fraise, bonbon', case=False)][['id_produit', 'saveur']]

# Afficher les produits_id filtrés
print(filtered_produit_id)


     id_produit          saveur
842         844  fraise, bonbon


In [ ]:
# Simuler les préférences de l'utilisateur (par exemple, un ensemble d'IDs de produits pertinents)
user_relevant_ids = [844]  # Exemple : ces produits sont pertinents pour l'utilisateur


In [127]:
query = "Quel produit contient des saveurs de fraise et bonbon ?"

In [128]:
# Fonction pour extraire les saveurs de la requête et les rechercher dans les résultats
def extract_flavors_from_query(query):
    """
    Cette fonction extrait les saveurs présentes dans la requête `query`
    et retourne une liste des saveurs trouvées parmi celles définies dans `flavor_list`.
    """
    # Liste des saveurs disponibles
    flavor_list = ["cassis", "lime", "fraise", "menthe", "mangue", "pêche", "bonbon"]

    found_flavors = []
    
    # Pour chaque saveur de la liste, vérifier si elle est mentionnée dans la requête
    for flavor in flavor_list:
        if re.search(rf'\b{flavor}\b', query, re.IGNORECASE):
            found_flavors.append(flavor)

    return found_flavors


# Fonction pour interroger le vecteur et récupérer les saveurs associées à la requête
def search_for_flavor(query):
    # Extraire les saveurs mentionnées dans la requête
    flavors_in_query = extract_flavors_from_query(query)

    # Afficher uniquement les saveurs extraites
    print(f"Saveurs extraites de la requête : {flavors_in_query}")

    # Retourner les saveurs extraites sous forme de chaîne, séparées par des virgules
    return ", ".join(flavors_in_query)



# Utilisation de la fonction search_for_flavor dans retriever.invoke
flavors_in_query = search_for_flavor(query)

Saveurs extraites de la requête : ['fraise', 'bonbon']


In [119]:
flavors_in_query

'fraise, bonbon'

In [141]:
# Récupérer les recommandations du modèle
recommendations = retriever.invoke(query,  filter={"saveur": flavors_in_query}) #, filter={"saveur": flavors_in_query})  # Exécution du retriever avec une requête d'exemple
recommended_ids = [rec.metadata["id_produit"] for rec in recommendations]


In [143]:
recommended_ids

[844]

In [142]:
recommendations

[Document(metadata={'source': '../../RGBD/table_produits/produits.csv', 'row': 842, 'id_produit': 844, 'saveur': 'fraise, bonbon'}, page_content="Le Fraise Candy est un e liquide 50ml d'origine française, fabriqué par Prestige Fruits (Vape Connection) pour la gamme Sweety Fruits. Il vous propose une saveur fraîche de fraise façon bonbon. Créé à partir d'arômes, de propylène glycol (PG) et de glycérine végétale (VG), il est proposé en50PG/50VGafin d'offrir un excellent compromis entre saveurs, hit en gorge et vapeur. Il est compatible tout type de matériel e-cig. Le Fraise Candy est un e liquide surboosté et non nicotiné, conditionné en fiole de type PET 60ml. Dessus, un bouchon avec sécurité enfant et un embout fin très pratique. Rempli à hauteur de 50ml, il pourra être accompagné d'un booster de nicotine au besoin. Qui est Prestige Fruits ? Comme son nom l'indique, le fabriquant française s'est spécialisé dans la reproduction des saveurs de fruits à travers sa large gamme d'e-liquides

In [122]:
retriever.invoke(query,  filter={"saveur": flavors_in_query})

[Document(metadata={'source': '../../RGBD/table_produits/produits.csv', 'row': 842, 'id_produit': 844, 'saveur': 'fraise, bonbon'}, page_content="Le Fraise Candy est un e liquide 50ml d'origine française, fabriqué par Prestige Fruits (Vape Connection) pour la gamme Sweety Fruits. Il vous propose une saveur fraîche de fraise façon bonbon. Créé à partir d'arômes, de propylène glycol (PG) et de glycérine végétale (VG), il est proposé en50PG/50VGafin d'offrir un excellent compromis entre saveurs, hit en gorge et vapeur. Il est compatible tout type de matériel e-cig. Le Fraise Candy est un e liquide surboosté et non nicotiné, conditionné en fiole de type PET 60ml. Dessus, un bouchon avec sécurité enfant et un embout fin très pratique. Rempli à hauteur de 50ml, il pourra être accompagné d'un booster de nicotine au besoin. Qui est Prestige Fruits ? Comme son nom l'indique, le fabriquant française s'est spécialisé dans la reproduction des saveurs de fruits à travers sa large gamme d'e-liquides

In [109]:
retriever.invoke(query,  filter={"saveur": 'fraise, bonbon'})

[Document(metadata={'source': '../../RGBD/table_produits/produits.csv', 'row': 842, 'id_produit': 844, 'saveur': 'fraise, bonbon'}, page_content="Le Fraise Candy est un e liquide 50ml d'origine française, fabriqué par Prestige Fruits (Vape Connection) pour la gamme Sweety Fruits. Il vous propose une saveur fraîche de fraise façon bonbon. Créé à partir d'arômes, de propylène glycol (PG) et de glycérine végétale (VG), il est proposé en50PG/50VGafin d'offrir un excellent compromis entre saveurs, hit en gorge et vapeur. Il est compatible tout type de matériel e-cig. Le Fraise Candy est un e liquide surboosté et non nicotiné, conditionné en fiole de type PET 60ml. Dessus, un bouchon avec sécurité enfant et un embout fin très pratique. Rempli à hauteur de 50ml, il pourra être accompagné d'un booster de nicotine au besoin. Qui est Prestige Fruits ? Comme son nom l'indique, le fabriquant française s'est spécialisé dans la reproduction des saveurs de fruits à travers sa large gamme d'e-liquides

In [110]:
# Calculer la précision
def calculate_precision(recommended_ids, relevant_ids):
    # Nombre de recommandations pertinentes
    relevant_recommendations = set(recommended_ids).intersection(set(relevant_ids))
    precision = len(relevant_recommendations) / len(recommended_ids) if len(recommended_ids) > 0 else 0
    return precision

In [111]:
# Calculer la précision pour les recommandations données
precision = calculate_precision(recommended_ids, user_relevant_ids)

In [112]:
print(f"Précision (Precision) : {precision * 100:.2f}%")

Précision (Precision) : 100.00%


precision

recuperer le saveur

In [ ]:


# Fonction pour extraire les saveurs de la requête
def extract_flavors_from_query(query):
    flavor_list = ["cassis", "lime", "fraise", "menthe", "mangue", "pêche", "bonbon"]
    found_flavors = []

    # Pour chaque saveur dans la liste, vérifier si elle est mentionnée dans la requête
    for flavor in flavor_list:
        if re.search(rf'\b{flavor}\b', query, re.IGNORECASE):
            found_flavors.append(flavor)

    return found_flavors

# Fonction principale de recherche avec filtrage post-retrieval
def search_for_flavor_with_other_flavors(query, vectorstore):
    # Extraire les saveurs mentionnées dans la requête
    flavors_in_query = extract_flavors_from_query(query)

    if not flavors_in_query:
        print("Aucune saveur mentionnée dans la requête.")
        return []

    print(f"Saveurs extraites de la requête : {flavors_in_query}")

    # Paramétrer le retriever pour la recherche par similarité
    retriever = vectorstore.as_retriever(
        search_type="similarity",  # Recherche des voisins les plus proches
        search_kwargs={
            "k": 10,  # Retourner les 10 voisins les plus proches
        }
    )

    # Effectuer la recherche dans le vectorstore
    results = retriever.invoke(query)

    # Filtrage des résultats pour ne garder que ceux contenant les saveurs mentionnées
    filtered_results = []
    for result in results:
        # Vérifier si la saveur de la requête est présente dans les résultats
        product_flavors = result.metadata.get('saveur', '').lower().split(', ')
        if any(flavor.lower() in product_flavors for flavor in flavors_in_query):
            filtered_results.append(result)

    # Si aucun produit n'est trouvé avec les saveurs de la requête, renvoyer tous les résultats
    if not filtered_results:
        filtered_results = results

    # Retourner les résultats filtrés
    return filtered_results


In [176]:
query = "Quel produit contient des saveurs fraise et bonbon ?"

affiche les saveurs

In [177]:
# Appeler la fonction de recherche
results = search_for_flavor_with_other_flavors(query, vectorstore)

Saveurs extraites de la requête : ['fraise', 'bonbon']


affiche les id des produits recommandés

In [179]:
recommended_ids = [rec.metadata["id_produit"] for rec in results]
recommended_ids


[855, 464, 797, 844, 921, 822]

a la main mettre en place une variable avec le id du produit qui match mieux

In [187]:
user_relevant_ids = [844, 921, 797, 464, 855]

calculer la precision

In [188]:
# Calculer la précision
def calculate_precision(recommended_ids, relevant_ids):
    # Nombre de recommandations pertinentes
    relevant_recommendations = set(recommended_ids).intersection(set(relevant_ids))
    precision = len(relevant_recommendations) / len(recommended_ids) if len(recommended_ids) > 0 else 0
    return precision

# Calculer la précision pour les recommandations données
precision = calculate_precision(recommended_ids, user_relevant_ids)
print(f"Précision (Precision) : {precision * 100:.2f}%")

Précision (Precision) : 83.33%


In [182]:


# Assurez-vous que vectorstore est correctement initialisé et contient les bons produits

# Appeler la fonction de recherche
results = search_for_flavor_with_other_flavors(query, vectorstore)

# Afficher les résultats
if results:
    for result in results:
        print(result.metadata)
else:
    print("Aucun produit correspondant.")

Saveurs extraites de la requête : ['fraise', 'bonbon']
{'source': '../../RGBD/table_produits/produits.csv', 'row': 853, 'id_produit': 855, 'saveur': 'parfait, pomme, bonbon, bonbons'}
{'source': '../../RGBD/table_produits/produits.csv', 'row': 462, 'id_produit': 464, 'saveur': 'fraise, barbe à papa, lime, bonbon, bonbons'}
{'source': '../../RGBD/table_produits/produits.csv', 'row': 795, 'id_produit': 797, 'saveur': 'fraise, lime, bonbon, fruit du dragon'}
{'source': '../../RGBD/table_produits/produits.csv', 'row': 842, 'id_produit': 844, 'saveur': 'fraise, bonbon'}
{'source': '../../RGBD/table_produits/produits.csv', 'row': 919, 'id_produit': 921, 'saveur': 'fraise, lime, vin, bonbon'}
{'source': '../../RGBD/table_produits/produits.csv', 'row': 820, 'id_produit': 822, 'saveur': 'fraise, lime, figue'}


In [185]:
flavors_in_query

['fraise', 'bonbon']

In [138]:
# Requête de l'utilisateur
query = "Quel produit contient des saveurs fraise et bonbon ?"


# Appeler la fonction de recherche
results = search_for_flavor_with_other_flavors(query, vectorstore)

# Afficher les résultats
if results:
    print("Produits trouvés :", results)
else:
    print("Aucun produit correspondant.")


Saveurs extraites de la requête : ['fraise', 'bonbon']
Aucun produit correspondant.


In [153]:
def extract_flavors_from_query(query):
    """
    Cette fonction extrait les saveurs présentes dans la requête `query`
    et retourne une liste des saveurs trouvées parmi celles définies dans `flavor_list`.
    """
    # Liste des saveurs disponibles
    flavor_list = ["cassis", "lime", "fraise", "menthe", "mangue", "pêche", "bonbon"]

    found_flavors = []
    
    # Pour chaque saveur de la liste, vérifier si elle est mentionnée dans la requête
    for flavor in flavor_list:
        if re.search(rf'\b{flavor}\b', query, re.IGNORECASE):
            found_flavors.append(flavor)

    return found_flavors

def search_for_flavor_with_other_flavors(query, vectorstore):
    # Extraire les saveurs mentionnées dans la requête
    flavors_in_query = extract_flavors_from_query(query)

    if not flavors_in_query:
        print("Aucune saveur mentionnée dans la requête.")
        return []

    print(f"Saveurs extraites de la requête : {flavors_in_query}")

    # Si moins de 2 saveurs sont mentionnées, utiliser un filtre strict
    if len(flavors_in_query) < 2:
        retriever = vectorstore.as_retriever(
            search_type="similarity",  # Recherche des voisins les plus proches
            search_kwargs={
                "k": 10,  # Retourner les 10 voisins les plus proches
            }
        )

        # Effectuer la recherche dans le vectorstore avec un filtre sur les saveurs
        results = retriever.invoke(query, filter={"saveur": flavors_in_query})

        return results
    
    # Si 2 ou plus saveurs sont mentionnées, effectuer une recherche par similarité puis filtrer les résultats
    elif len(flavors_in_query) >= 2:
        retriever = vectorstore.as_retriever(
            search_type="similarity",  # Recherche des voisins les plus proches
            search_kwargs={
                "k": 10,  # Retourner les 10 voisins les plus proches
            }
        )

        # Effectuer la recherche dans le vectorstore
        results = retriever.invoke(query)

        # Filtrage des résultats pour ne garder que ceux contenant les saveurs mentionnées
        filtered_results = []
        for result in results:
            product_flavors = result.metadata.get('saveur', '').lower().split(', ')
            if any(flavor.lower() in product_flavors for flavor in flavors_in_query):
                filtered_results.append(result)

        # Retourner les résultats filtrés
        return filtered_results


In [154]:
query = "Quel produit contient des saveurs de fraise et de bonbon ?"

In [155]:
flavors_in_query = extract_flavors_from_query(query)

In [156]:
flavors_in_query

['fraise', 'bonbon']

In [157]:


# Assurez-vous que vectorstore est correctement initialisé et contient les bons produits

# Appeler la fonction de recherche
results = search_for_flavor_with_other_flavors(query, vectorstore)

# Afficher les résultats
if results:
    for result in results:
        print(result.metadata)
else:
    print("Aucun produit correspondant.")

Saveurs extraites de la requête : ['fraise', 'bonbon']
{'source': '../../RGBD/table_produits/produits.csv', 'row': 853, 'id_produit': 855, 'saveur': 'parfait, pomme, bonbon, bonbons'}
{'source': '../../RGBD/table_produits/produits.csv', 'row': 795, 'id_produit': 797, 'saveur': 'fraise, lime, bonbon, fruit du dragon'}
{'source': '../../RGBD/table_produits/produits.csv', 'row': 462, 'id_produit': 464, 'saveur': 'fraise, barbe à papa, lime, bonbon, bonbons'}
{'source': '../../RGBD/table_produits/produits.csv', 'row': 842, 'id_produit': 844, 'saveur': 'fraise, bonbon'}
{'source': '../../RGBD/table_produits/produits.csv', 'row': 919, 'id_produit': 921, 'saveur': 'fraise, lime, vin, bonbon'}
{'source': '../../RGBD/table_produits/produits.csv', 'row': 294, 'id_produit': 295, 'saveur': 'fraise, lime'}


In [158]:
uery = "Quel produit contient des saveurs de fraise ?"

# Assurez-vous que vectorstore est correctement initialisé et contient les bons produits

# Appeler la fonction de recherche
results = search_for_flavor_with_other_flavors(query, vectorstore)

# Afficher les résultats
if results:
    for result in results:
        print(result.metadata)
else:
    print("Aucun produit correspondant.")

Saveurs extraites de la requête : ['fraise', 'bonbon']
{'source': '../../RGBD/table_produits/produits.csv', 'row': 853, 'id_produit': 855, 'saveur': 'parfait, pomme, bonbon, bonbons'}
{'source': '../../RGBD/table_produits/produits.csv', 'row': 795, 'id_produit': 797, 'saveur': 'fraise, lime, bonbon, fruit du dragon'}
{'source': '../../RGBD/table_produits/produits.csv', 'row': 462, 'id_produit': 464, 'saveur': 'fraise, barbe à papa, lime, bonbon, bonbons'}
{'source': '../../RGBD/table_produits/produits.csv', 'row': 842, 'id_produit': 844, 'saveur': 'fraise, bonbon'}
{'source': '../../RGBD/table_produits/produits.csv', 'row': 919, 'id_produit': 921, 'saveur': 'fraise, lime, vin, bonbon'}
{'source': '../../RGBD/table_produits/produits.csv', 'row': 294, 'id_produit': 295, 'saveur': 'fraise, lime'}
